In [ ]:
R.home()

In [ ]:
baizer::pkglib(baizer, tidyverse, yaml)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])

In [ ]:
Pinput <- snakemake@input$filter_input

In [ ]:
# concat
Ltb <- str_c(Pinput, '/', basename(Pinput), '_stat.yaml') %>%
    map(~read_yaml(.x) %>% as_tibble)

Ltb <- map2(Ltb, basename(Pinput), 
     function(x, y) {
         rename_at(x, colnames(x), ~str_c(.x, '_', y))
     }
)

TBstat <- reduce(Ltb, bind_cols) %>% mutate(sample=sample, .before=1)

In [ ]:
# input cell
TBmeta <- read_csv(snakemake@params[['metadata']])
TBmeta_smp <- TBmeta %>% filter(sample==.env[['sample']]) 

Ncell <- TBmeta_smp[['input cells']]
if (length(Ncell) == 1) {
    TBstat <- TBstat %>% mutate(cells_input=Ncell, .after=sample)
}

In [ ]:
# relocate
TBstat <- TBstat %>% relocate(sample, contains('cells_'), contains('_VDJB'), 
                    contains('_FB'), contains('_mRNA'), contains('_VDJT'))

In [ ]:
TBstat %>% write_excel_csv(snakemake@output[['stats']])